# Check Score

In [1]:
import numpy as np
import pandas as pd

import os
import glob, datetime

In [2]:
BASE_DIR = os.path.join(os.getcwd(), os.pardir)
DATA_DIR = os.path.join(BASE_DIR, 'data')
OUTPUT_DIR = os.path.join(BASE_DIR, 'output')

TYPE_LABEL = {'clicks':0, 'carts':1, 'orders':2}

## ファイルimport

In [12]:
def read_file_to_cache(f):
    df = pd.read_parquet(f)
    df.ts = (df.ts/1000).astype('int32')
    df['type'] = df['type'].map(TYPE_LABEL).astype('int8')
    return df

In [74]:
%%time
# # ================
# # sample_submission
# # ================
path_sample_submission = os.path.join(BASE_DIR, 'data', 'original_data', 'sample_submission.csv')
sample_sub = pd.read_csv(path_sample_submission)


# # ================
# # train/test parquet
# # ================
# train_paths = glob.glob(os.path.join(DATA_DIR, 'chunked_data', 'train_parquet', '*'))
# test_paths = glob.glob(os.path.join(DATA_DIR, 'chunked_data', 'test_parquet', '*'))
# # data_cache_train = {}
# # data_cache_test = {}
# # for p in train_paths: data_cache_train[p] = read_file_to_cache(p)
# # for p in test_paths: data_cache_test[p] = read_file_to_cache(p)

# ================
# validation file
# ================
train_val_paths = glob.glob(os.path.join(DATA_DIR, 'validation', 'train_parquet', '*'))
test_val_paths = glob.glob(os.path.join(DATA_DIR, 'validation', 'test_parquet', '*'))
test_label = os.path.join(DATA_DIR, 'validation', 'test_labels.parquet')
validation_train = {}
validation_test = {}
for p in train_val_paths: validation_train[p] = read_file_to_cache(p)
for p in test_val_paths: validation_test[p] = read_file_to_cache(p)
validation_label = pd.read_parquet(test_label)

CPU times: total: 3.39 s
Wall time: 3.41 s


In [81]:
# train/test
display( list( data_cache_train.values() )[0].sample() )
display( list( data_cache_test.values() )[0].sample() )

# validation
display( list( validation_train.values() )[0].sample() )
display( list( validation_test.values() )[0].sample() )

# sample submission


,session,aid,ts,type
2442641,46554,562295,1661344534,0


,session,aid,ts,type
137246,12926064,442580,1661760349,0


,session,aid,ts,type
732402,14839,585934,1660418590,0


,session,aid,ts,type
181091,11132579,1059808,1661151349,0


## Check Score

- param3
    - validation
    - 過去activityありaidのみ返す

In [3]:
param = 'param3'
target = 'validation'

result_dir = os.path.join(OUTPUT_DIR, target, 'result', param)

In [20]:
def _read_result(fname):
    path = os.path.join(result_dir, fname)
    if os.path.splitext(path)[-1] == '.parquet':
        df = pd.read_parquet(path)
    elif os.path.splitext(path)[-1] == '.csv':
        df = pd.read_csv(path)
    return df

# fname = 'prediction_carts.parquet'
# fname = 'prediction_orders.parquet'
# fname = 'score_detail_carts.parquet'
fname = 'score_detail_orders.parquet'

# fname = 'prediction.csv'
# fname = 'score_all.csv'

df = _read_result(fname)

In [24]:
# df.label_count.max()
df

,session,type,ground_truth,session_type,labels,hits,gt_count,label_count
0,11098528,orders,"[990658, 950341, 1462506, 1561739, 907564, 369...",11098528_orders,[11830],1,11,1
1,11098530,orders,[409236],11098530_orders,"[409236, 264500]",1,1,2
2,11098531,orders,[1365569],11098531_orders,"[396199, 1271998, 452188, 1728212, 1365569, 62...",1,1,11
3,11098533,orders,"[935297, 652916, 334581, 1233050, 1366413, 118...",11098533_orders,"[1165015, 385390, 978918, 1074173, 765030, 833...",1,6,13
4,11098534,orders,[223062],11098534_orders,"[908024, 223062, 1342293, 1607945, 1449202]",1,1,5
...,...,...,...,...,...,...,...,...
150812,12899329,orders,[931182],12899329_orders,[1333457],0,1,1
150813,12899337,orders,[851751],12899337_orders,[558573],0,1,1
150814,12899355,orders,"[1115942, 465366]",12899355_orders,[933686],0,2,1
150815,12899373,orders,[1766353],12899373_orders,"[1766353, 995962, 487949]",1,1,3


In [15]:
# df['each_score'] = df['hits'] / df['gt_count']

,session,type,ground_truth,session_type,labels,hits,gt_count,label_count
0,11098528,orders,"[990658, 950341, 1462506, 1561739, 907564, 369...",11098528_orders,[11830],1,11,1
1,11098530,orders,[409236],11098530_orders,"[409236, 264500]",1,1,2
2,11098531,orders,[1365569],11098531_orders,"[396199, 1271998, 452188, 1728212, 1365569, 62...",1,1,11
3,11098533,orders,"[935297, 652916, 334581, 1233050, 1366413, 118...",11098533_orders,"[1165015, 385390, 978918, 1074173, 765030, 833...",1,6,13
4,11098534,orders,[223062],11098534_orders,"[908024, 223062, 1342293, 1607945, 1449202]",1,1,5
...,...,...,...,...,...,...,...,...
150812,12899329,orders,[931182],12899329_orders,[1333457],0,1,1
150813,12899337,orders,[851751],12899337_orders,[558573],0,1,1
150814,12899355,orders,"[1115942, 465366]",12899355_orders,[933686],0,2,1
150815,12899373,orders,[1766353],12899373_orders,"[1766353, 995962, 487949]",1,1,3
